In [ ]:
import pandas as pd
import numpy as np

import time
import re

pd.set_option('display.max_colwidth', None)

In [ ]:
from py2neo import Graph
import pandas as pd
import numpy as np

from datetime import date
import datetime as dt
import time
import re

from bs4 import BeautifulSoup

import progressbar

pd.set_option('display.max_columns', None)

In [ ]:
# add geckodriver path
import sys
path = '/Users/catarina/Projects/bin'
sys.path.append(path)

# 755 is the default numerical permission for files in usr/bin
# webdriver needs a numerical permission equivalent to or greater than 755
import os
os.chmod(path,755) #664

from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.options import Options

# torexe = os.popen(r'C:\Users\AtechM_03\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')
torexe = os.popen('/usr/local/Cellar/tor/0.4.0.5_1/bin/tor.exe')

# profile = FirefoxProfile(r'C:\Users\AtechM_03\Desktop\Tor Browser\B/rowser\TorBrowser\Data\Browser\profile.default')
profile = FirefoxProfile('/Users/catarina/Library/Application Support/TorBrowser-Data/Browser/4dgz6wzb.default')

profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", False)
profile.update_preferences()

# don't open browser
options = Options()
options.add_argument("--headless")

In [ ]:
# Check tor is working

driver = webdriver.Firefox(firefox_profile = profile, executable_path = path + '/geckodriver')
driver.get("http://check.torproject.org")

In [ ]:
def get_soup_content(website):
        
    driver = webdriver.Firefox(options = options, 
                               firefox_profile = profile, 
                               executable_path = path + '/geckodriver')
    
    driver.get(website)

    content = driver.page_source
    
    soup = BeautifulSoup(content)
    
    return soup



def get_all_urls_for_tv_show(soup):
    
    urls_index = []

    for url in soup.findAll('a'):

        url = url['href']

        if (url.startswith('./viewforum')) and ('&start=' in url):

            url = 'http://transcripts.foreverdreaming.org' + url[1:].split('&')[0] + '&' + url.split('&')[-1]

            if url not in urls_index:
                
                urls_index.append(url)
                
    return urls_index


def get_episodes_urls(soup):

    episodes = []
    urls = []

    for item in soup.findAll('a', attrs={'class': 'topictitle'}):

        if item.text.startswith('Board Updates') or item.text.startswith('Online Store'):
            continue

        url = 'http://transcripts.foreverdreaming.org' + item['href'][1:].split('&sid')[0]

        episodes.append(item.text)
        urls.append(url)
        
    return pd.DataFrame({'episode': episodes, 'url': urls})



def get_script_tags(soup):
    
    script_tags = []
    
    for p in soup.findAll(['p', 'hr']):
        if len(p.attrs) > 0:
            continue
        if p.find('a'):
            continue
        script_tags.append(p)
        
    return script_tags



def parse_script(script_tags):

    # initialize counters
    scene = 1
    dialogue = 1
    sequence = 1

    scenes = []
    dialogues = []
    sequences = []
    characters = []
    lines = []

    for tag in script_tags:

        content = tag.text
        
        # exclude content between parenthesis and brackets (non verbal information)
        content = re.sub("[\(\[].*?[\)\]]", "", content.strip())
        
        # check if content is speech
        is_speech = ":" in content

                    
        # check if there is a change of dialogue (non dialogue text is interpreted as a change of context)
        if (is_speech == False) & (sequence > 1):
            dialogue += 1
            sequence = 1
            continue
            
        elif len(content) == 0:
            continue

        
        # parse the text by splitting character and speech on the ':' 
        # e.g. of text ("Chandler: Could this be any more amusing?")
        if is_speech:

            character = content.split(':')[0].strip()
            speech = " ".join(content.split(':')[1:])
            speech = speech.strip()


            scenes.append(scene)
            dialogues.append(dialogue)
            sequences.append(sequence)
            characters.append(character)
            lines.append(speech)


            sequence += 1

    
    df = pd.DataFrame({'scene': scenes, 
                       'dialogues': dialogues,
                       'sequence': sequences, 
                       'character': characters, 
                       'line': lines})
        
    return df



def scrape_list_of_episodes(tv_show_url):
    
    soup = get_soup_content(tv_show_url)
    
    
    # get the urls from all the episodes
    
    other_urls = get_all_urls_for_tv_show(soup) # get all page urls with episodes from tv show
    
    episodes = get_episodes_urls(soup) # get episodes from current page
    
    
    for url in other_urls: # get episodes listed in other pages
        
        soup = get_soup_content(url)
        
        add_episodes = get_episodes_urls(soup)
        
        episodes = pd.concat([episodes, add_episodes], axis = 0, ignore_index = True)
     
    
    return episodes
        

def scrape_dialogues(episodes):
        
    # get dialogues for each episode
        
    bar = progressbar.ProgressBar(maxval = episodes.shape[0],
                                  widgets = [progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    
    bar.start()
    
    
    try:
    
        dialogues = pd.DataFrame()
        
        i = 0

        for index, row in episodes.iterrows():

            soup = get_soup_content(row.url)

            script_tags = get_script_tags(soup)

            dialogue_ep = parse_script(script_tags)

            dialogue_ep['episode'] = row.episode

            dialogues = pd.concat([dialogues, dialogue_ep], axis = 0, ignore_index = True) 
            
            i += 1
            
            bar.update(i)
                
            time.sleep(3)
            
            
    except Exception as ex:

        print(ex)
        
        return index, row.url, dialogues
    
    return dialogues

In [ ]:
url_ = 'http://transcripts.foreverdreaming.org/viewforum.php?f=22'

# scrape the urls of all episodes

episodes = scrape_list_of_episodes(url_)
episodes = episodes[episodes.episode != 'Gilmore Girls Transcript Index']
episodes['season'] = episodes.episode.apply(lambda t: int(t.split('x')[0]))

In [ ]:
# scrape the dialogues from the episodes

dialogues = scrape_dialogues(episodes)

In [ ]:
# in case there is a timeout in the requests from the function above, recover the dialogues scraped so far

error_index, error_row, error_dialogues = dialogues